In [ ]:
import bs4
import requests
import pandas as pd
import numpy as np
import time
import os

In [ ]:
#구글 드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def getSource(url):

    # 헤더 정보 셋팅
    header_info = {
        'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.5249.114 Whale/3.17.145.12 Safari/537.36',
    }
    # 요청한다.
    response = requests.get(url, headers=header_info)
    return response.text

In [ ]:
def getData(res, fp) :
    # BeautifulSoup 객체 생성
    soup = bs4.BeautifulSoup(res, 'html.parser')

    # 플레이리스트 제목, 작성자, 곡수 추출
    BASE_DESCRIPTION = "Spotify 플레이리스트 ["
    PLAYLIST_TITLE = soup.select_one('h1').text.strip()
    PLAYLIST_OWNER = soup.select_one('div.Zlw_JpvCycfWtpw7K5Oq').text.strip()
    SONG_NUM = len(soup.select('span.fYiTde'))
    NEW_LINE = BASE_DESCRIPTION + PLAYLIST_TITLE + "] / 작성자: " + PLAYLIST_OWNER + " / " + str(SONG_NUM) + "곡\n"
    fp.write(NEW_LINE)

    # 플레이리스트를 포함하고 있는 2번째 div를 선택
    targets = soup.select_one('div.JUa6JJNj7R_Y3i4P8YUX > div:nth-of-type(2)')
    result = []
    for target in targets:
      title = target.select_one('a.cGeiev').text.strip()
      artists = target.select('span.fMlCXO')
      for artist in artists:
        artist = artist.text.strip()
        result.append([title, artist])
    
    return result

In [ ]:
emotionDict = {
    1: 'SAD',
    2: 'MILD',
    3: 'NATURE',
    4: 'LOUD',
    5: 'SENTIMENTAL',
    6: 'FUNNY',
    7: 'LOVELY',
}
emotion = emotionDict[1]

url = 'https://open.spotify.com/playlist/37i9dQZF1DXb5KQr73mtTz'
WORKSPACE_ROUTE = '/content/drive/My Drive/Colab Notebooks/(AI_Project) unemployedStation/'
DESCRIPTION_ROUTE = WORKSPACE_ROUTE + 'data/description/'+ emotion + '_DESCRIPTION.txt'
PLAYLIST_ROUTE = WORKSPACE_ROUTE + 'data/playlist/'+ emotion + '_PLAYLIST.csv'

if not os.path.exists(DESCRIPTION_ROUTE):
  fp = open(DESCRIPTION_ROUTE, "w")
else:
  fp = open(DESCRIPTION_ROUTE, "a")

# 요청한다
res = getSource(url)
result = getData(res, fp)

fp.close()
df = pd.DataFrame(result, columns = ['title', 'artist'])

if not os.path.exists(PLAYLIST_ROUTE):
  df.to_csv(PLAYLIST_ROUTE, index=False, mode='w', encoding='utf-8-sig')
else:
  df.to_csv(PLAYLIST_ROUTE, index=False, mode='a', encoding='utf-8-sig', header=False)

display(df.head(5))
display(df.tail(5))

,title,artist
0,Epiphany,BTS
1,I Will Go To You Like the First Snow,AILEE
2,This Love,DAVICHI
3,Ending Scene,IU
4,Me After You,Paul Kim


,title,artist
45,Memory of You,gyuri
46,girl in red,ALEPH
47,Worse than friends,Jeong Dong Won
48,How Can I Live Without You,Juho
49,Dimly,RIO
